# ML2025 Homework 1 - Retrieval Augmented Generation with Agents

## Environment Setup

In this section, we install the necessary python packages and download model weights of the quantized version of LLaMA 3.1 8B. Also, download the dataset. Note that the model weight is around 8GB.

In [34]:
!python3 -m pip install --no-cache-dir llama-cpp-python==0.3.4 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122
!python3 -m pip install googlesearch-python bs4 charset-normalizer requests-html lxml_html_clean

from pathlib import Path
if not Path('./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf').exists():
    !wget https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf
if not Path('./public.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/public.txt
if not Path('./private.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/private.txt

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122


In [35]:
import torch
if not torch.cuda.is_available():
    raise Exception('You are not using the GPU runtime. Change it first or you will suffer from the super slow inference speed!')
else:
    print('You are good to go!')

You are good to go!


## Prepare the LLM and LLM utility function

By default, we will use the quantized version of LLaMA 3.1 8B. you can get full marks on this homework by using the provided LLM and LLM utility function. You can also try out different LLM models.

In the following code block, we will load the downloaded LLM model weights onto the GPU first.
Then, we implemented the generate_response() function so that you can get the generated response from the LLM model more easily.

You can ignore "llama_new_context_with_model: n_ctx_per_seq (16384) < n_ctx_train (131072) -- the full capacity of the model will not be utilized" warning.

In [36]:
from llama_cpp import Llama

# Load the model onto GPU
llama3 = Llama(
    "./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf",
    verbose=False,
    n_gpu_layers=-1,
    n_ctx=16384,    # This argument is how many tokens the model can take. The longer the better, but it will consume more memory. 16384 is a proper value for a GPU with 16GB VRAM.
)

def generate_response(_model: Llama, _messages: str) -> str:
    '''
    This function will inference the model with given messages.
    '''
    _output = _model.create_chat_completion(
        _messages,
        stop=["<|eot_id|>", "<|end_of_text|>"],
        max_tokens=512,    # This argument is how many tokens the model can generate.
        temperature=0,      # This argument is the randomness of the model. 0 means no randomness. You will get the same result with the same input every time. You can try to set it to different values.
        repeat_penalty=2.5,
    )["choices"][0]["message"]["content"]
    return _output

llama_new_context_with_model: n_ctx_per_seq (16384) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


In [37]:
# You can try out different questions here.
test_question='請問誰是 G-Dragon？'

messages = [
    {"role": "system", "content": "你是 LLaMA-3.1-8B，是用來回答問題的 AI。使用中文時只會使用繁體中文來回問題。"},    # System prompt
    {"role": "user", "content": test_question}, # User prompt
]

print(generate_response(llama3, messages))

G-Dragon 是韓國男子偶像團體BIGBANG的主唱兼領舞。他的本名是金大賢（Kwon Ji-yong），出生於1988年，來自南京道光州廣域市。他以其獨特且創新的音樂風格、時尚感和個人魅力而聞name。

G-Dragon 是韓國流行文化的一部分，他的作品不僅影響了K-pop，也對全球化潮牌業界有所貢獻。他的歌曲如《Heartbreaker》、《Butterfly》，以及與其他BIGBANG成員合作創作的大膽且前衛的小說般音樂錄影帶，都深受粉絲和樂迷的喜愛。

除了是藝人外，G-Dragon 也是一位成功企業家，他成立了自己的品牌YG Entertainment旗下的子公司《ADOR》以及潮牌「PEACEMINUSONE」，並與全球知名設計師合作推出多款時尚產品。


## Search Tool

The TA has implemented a search tool for you to search certain keywords using Google Search. You can use this tool to search for the relevant **web pages** for the given question. The search tool can be integrated in the following sections.

In [ ]:
from typing import List
from googlesearch import search as _search
from bs4 import BeautifulSoup
from charset_normalizer import detect
import asyncio
from requests_html import AsyncHTMLSession
import urllib3
urllib3.disable_warnings()

async def worker(s:AsyncHTMLSession, url:str):
    try:
        header_response = await asyncio.wait_for(s.head(url, verify=False), timeout=10)
        if 'text/html' not in header_response.headers.get('Content-Type', ''):
            return None
        r = await asyncio.wait_for(s.get(url, verify=False), timeout=10)
        
        # ✏️ Check for PDF signatures in the content
        if r.text.startswith('%PDF'):
            return None
        return r.text
    except:
        return None

async def get_htmls(urls):
    session = AsyncHTMLSession()
    tasks = (worker(session, url) for url in urls)
    return await asyncio.gather(*tasks)

async def search(keyword: str, n_results: int=3) -> List[str]:
    '''
    This function will search the keyword and return the text content in the first n_results web pages.
    Warning: You may suffer from HTTP 429 errors if you search too many times in a period of time. This is unavoidable and you should take your own risk if you want to try search more results at once.
    The rate limit is not explicitly announced by Google, hence there's not much we can do except for changing the IP or wait until Google unban you (we don't know how long the penalty will last either).
    '''
    keyword = keyword[:100]
    # First, search the keyword and get the results. Also, get 2 times more results in case some of them are invalid.
    results = list(_search(keyword, n_results * 2, lang="zh", unique=True))
    # Then, get the HTML from the results. Also, the helper function will filter out the non-HTML urls.
    results = await get_htmls(results)
    # Filter out the None values.
    results = [x for x in results if x is not None]
    # Parse the HTML.
    results = [BeautifulSoup(x, 'html.parser') for x in results]
    # Get the text from the HTML and remove the spaces. Also, filter out the non-utf-8 encoding.
    results = [''.join(x.get_text().split()) for x in results if detect(x.encode()).get('encoding') == 'utf-8']
    # ✏️ Return the truncated results.
    return "\n".join([f'[webpage {i} begin]{results[i][:3200]}...[webpage {i} end]' for i in range(min(n_results, len(results)))])

## Test the LLM inference pipeline

### For example

In [39]:
# ✏️ Let's look at the problem in the hand-crafted public dataset.
test_question='校歌為學校（包括小學、中學、大學等）宣告或者規定的代表該校的歌曲。用於體現該校的治學理念、辦學理想等學校文化。「虎山雄風飛揚」是哪間學校的校歌歌詞？'

messages = [
    {"role": "system", "content": "你是 LLaMA-3.1-8B，是用來回答問題的 AI。使用中文時只會使用繁體中文來回問題。"},    # System prompt
    {"role": "user", "content": test_question}, # User prompt
]

print(generate_response(llama3, messages))

"虎山雄風飛揚 " 是國立臺灣師範大學的校歌。


In [40]:
# ✏️ Suppose we already have an agent responsible for the keyword extraction
retrieval = await search(keyword = "虎山雄風飛揚 校歌")
retrieval

'[webpage 0 begin]校歌南投縣南投市光華國小-精華區lyrics-批踢踢實業坊批踢踢實業坊›精華區betalyrics關於我們聯絡資訊返回上層作者ypx0409(ypx0409)看板lyrics標題校歌南投縣南投市光華國小時間TueSep1603:13:342008貓羅溪旁虎山雄風飛揚這兒是我們生長的地方良師益友濟濟一堂克勤克儉奮發圖強長我育我飲水思源立定志向四海名揚忠心復國是理想光華意志堅光華氣勢昂你我相共勉誓把華夏重光--※發信站:批踢踢實業坊(ptt.cc)◆From:122.116.179.102推vibrant224:國小校歌推...09/1611:10推sognare:校友友情推09/1614:09...[webpage 0 end]\n[webpage 1 begin]走過一甲子跳到主要內容區您的瀏覽器不支援JavaScript功能，若網頁功能無法正常使用時，請開啟瀏覽器JavaScript狀態Menu回首頁Home網站導覽WebGuide關於光華AboutUs學校簡介校史沿革學校概況光華校旗光華校歌光華校景交通位置光華學生數光華一甲子60GloriousYears歷屆校長光華一甲子再創百年史走過一甲子資優教育史體育輝煌史音樂璀璨史藝文榮耀史光華校園誌歷史剪影60週年活動花絮行政處室OurTeam校長室教務處學務處輔導處總務處人事室會計室校務應用RelatedWebs校務行政系統南投縣教育處教師在職進修網WebITR差勤系統課務排代數位學習推動辦公室校園雲端電子信箱學習扶助科技評量啟用教育雲端帳號(OpenID)校園午餐登錄平台各類獎狀列印畢業獎項列印公文整合資訊系統會計系統校園花絮Photos電子相簿線上影音校內網站websites光華FB粉絲頁班級網站光華資優班特教研習及活動資訊不分類資源班光華交通安全網光華圖書室光華附設幼兒園南投縣資優資源教育中心校園導覽VR720自主學習NeverStopLearning校園學生資源網googleclassroom登入GoogleClassroom登入教學教育體系單一簽入服務教育部因材網PaGamO線上學習平台線上教學便利包康軒雲南一OneBox翰林行動大師真平本土語在家學何嘉仁ABC英語世界學習吧均一教育平台CoolEnglish線上學習平台Hami書城-校園電子書平台瓜瓜漂流網閱讀認證學校行

In [41]:
from datetime import datetime
cur_day = datetime.today().date()

messages = [
    {"role": "system", "content": "你是 LLaMA-3.1-8B，是用來回答問題的 AI。在使用中文時只會使用繁體中文來回問題。"},    # System prompt
    {"role": "user", "content": f"""

    以下內容是基於使用者發送的消息的搜尋結果:
    {retrieval}

    在我給你的搜尋結果中，每個結果都是[webpage X begin]...[webpage X end]格式的，X代表每篇文章的數字索引。請在適當的情況下在句子末尾引用上下文。請按照引用編號[citation:X]的格式在答案對應部分引用上下文。如果一句話源自多個上下文，請列出所有相關的引用編號，例如[citation:3][citation:5]，切記不要將引用集中在最後返回引用編號，而是在答案對應部分列出。 
    在回答時，請注意以下幾點： 
    - 今天是{cur_day}。 
    - 並非搜尋結果的所有內容都與使用者的問題密切相關，你需要結合問題，對搜尋結果進行甄別、篩選。
    - 如果回答很长，请尽量结构化、分段落总结。如果需要分点作答，尽量控制在5个点以内，并合并相关的内容。
    - 对于客观类的问答，如果问题的答案非常简短，可以适当补充一到两句相关信息，以丰富内容。
    - 你的回答应该综合多个相关网页来回答，不能重复引用一个网页。

    使用者訊息為：
    {test_question}"""}, # User prompt
]

print(generate_response(llama3, messages))

根據你的問題，我們可以在搜尋結果中找到相關的資訊。

[tiger mountain雄風飛揚] 是南投縣光華國小校歌的一部分。[citation:0]

以下是完整版：

貓羅溪旁虎山
　 雄风飞扬，这儿是我们生长的地方。
良师益友济濟一堂，
克勤 克儉奋发图强， 
長我育 我饮水思源立定志向四海名揚忠心复国是理想光华意誌坚 光華气勢昂你
　 们相共勵誓把 华夏重生--


### Tryout with the public dataset: Naive Approach

In [10]:
import asyncio

with open('./public.txt', 'r') as input_f:
    questions = input_f.readlines()
    questions = [l.strip().split(',')[0] for l in questions]
    # ✏️ We use the original question as the searching keyword.
    tasks = [search(keyword=q) for q in questions]
    results = await asyncio.gather(*tasks)
    
for id, (question, retrieval) in enumerate(zip(questions, results), 1):
    # prompt adapted from: https://github.com/deepseek-ai/DeepSeek-R1/issues/407
    messages = [
        {"role": "system", "content": "你是 LLaMA-3.1-8B，是用來回答問題的 AI。在使用中文時只會使用繁體中文來回問題。"},  # System prompt
        {"role": "user", "content": f"""
    
        以下內容是基於使用者發送的消息的搜尋結果:
        {retrieval}
    
        在我給你的搜尋結果中，每個結果都是[webpage X begin]...[webpage X end]格式的，X代表每篇文章的數字索引。請在適當的情況下在句子末尾引用上下文。請按照引用編號[citation:X]的格式在答案對應部分引用上下文。如果一句話源自多個上下文，請列出所有相關的引用編號，例如[citation:3][citation:5]，切記不要將引用集中在最後返回引用編號，而是在答案對應部分列出。 
        在回答時，請注意以下幾點： 
        - 今天是{cur_day}。 
        - 並非搜尋結果的所有內容都與使用者的問題密切相關，你需要結合問題，對搜尋結果進行甄別、篩選。
        - 如果回答很长，请尽量结构化、分段落总结。如果需要分点作答，尽量控制在5个点以内，并合并相关的内容。
        - 对于客观类的问答，如果问题的答案非常简短，可以适当补充一到两句相关信息，以丰富内容。
        - 你的回答应该综合多个相关网页来回答，不能重复引用一个网页。
    
        使用者訊息為：
        {question}"""},  # User prompt
    ]
    answer = generate_response(llama3, messages)
    answer = answer.replace('\n', ' ')
    print(id, answer)

1 根據你的問題，我們可以在搜尋結果中找到相關的資訊。[webpage 0 begin]和 [	webage	1	begin ] 都提到校歌是學校宣告或規定的代表該 校 的 歌曲， 用於體現 該 學院 或 大學 文化。  根據我的知識庫，我找到了關于「虎山雄風飛揚」的資訊。這首詩詞被用作多所大學的校歌，其中包括清華大学和新竹國立 清华大 学等學校。但是，沒有明確指出哪一間学校使用了这句诗作为其正式 校 歌。  如果你想知道更多關于「虎山雄風飛揚」的資訊，我建議可以在相關的網站或書籍中進行查詢。
2 根據搜尋結果，2025年初NCC透過行政命令規定民眾如果通過境外郵購無線鍑盤、滑鼠藍芽耳機等自用產品回台，每案一律加收審查費750元。  [citation:0][citation：1]
3 第一代 iPhone 是由史蒂夫·乔布斯發表的。[citation:1] 他在 2007 年的一月九日於 Macworld Conference & Expo 上正式介紹了這款智能手機，同年六 月二十 九 日開始銷售。  根據蘋果公司官方網站上的資訊，這部 iPhone 是史蒂夫·乔布斯的創意，他希望能夠將電腦和電話結合起來，使使用者可以輕鬆地瀏覽互聯 網絡、收發電子郵件等功能。[citation:2]  在蘋果公司內部，第一代 iPhone 的研製工作由一支團隊負責，他們花了近三年時間進行設計和開 發這款產品。在 2007 年的一月九日的 Macworld Conference & Expo 上，這個秘密計畫終於被公開。[citation:3]  史蒂夫·乔布斯在發表會上展示 iPhone 的時候，受到現場觀眾們熱烈歡迎，他說道：「這是一部革命性的產品，它將改變我們的生活方式。」 [ citation : 4 ]  第一代iPhone於2007年6月29日正式发售。
4 根據台大英文免修申請規定，托福網路測驗(TOEFL iBT)的總分需達到92或以上才能符合資格。[citation:2]  另外，也有提及其他相關條件，如多益、雅思等，但這裡主要是針對台大英文免修申請規定中的托福網路測驗要求進行回答。  如果您想了解更多關於台灣大學進階英文字母的資訊，建議可以參考以下內容：  * 台灣各個學校的大一新生都有機會通過優異英語成績來獲得免修或抵減學分。 例如：台大、政大的英文畢業門檻較

The answer is more sound though, even with a naive implementation of RAG. However, we can still find mistakes. Out of 30 questions, 12 answers (questions 1, 4, 6, 7, 9, 13, 14, 19, 22, 23, 25, and 28) are incorrect due to factual mismatches. Let us check them out. We quote the judgement from Qwen2.5 Max: https://chat.qwenlm.ai/s/5fb3e1e6-3b19-441e-8869-72fc154db35d

In [42]:
import re

# ✏️ Let's try extracting keywords
test_question='校歌為學校（包括小學、中學、大學等）宣告或者規定的代表該校的歌曲。用於體現該校的治學理念、辦學理想等學校文化。「虎山雄風飛揚」是哪間學校的校歌歌詞？'

messages = [
    {"role": "system", "content": "你是 LLaMA-3.1-8B，是一個善用網絡搜尋工具的AI，簡練，沒有廢話。"},    # System prompt
    {"role": "user", "content": f"""
    基於使用者發送的消息，過濾和問題無關信息，還原代詞，並提取出最相關且有用的關鍵詞，用於進行網絡搜索。
    
    input："《王者荣耀》是腾讯天美工作室推出的英雄竞技手游，不是一个人的王者，而是团队的荣耀！5v5王者峡谷PVP对战，领略英雄竞技的酣畅淋漓！那它现在是什么赛季？"
    output：抽取關鍵詞：```王者荣耀 赛季```

    input：{test_question}
    output："""}, # User prompt
]

generate_response(llama3, messages)

'抽取關鍵詞：```虎山雄風飛揚 校歌```\n\n我會根據使用者發送的消息，過濾和問題無相關信息，並提extract出最有用的資訊。'

## Agents

The TA has implemented the Agent class for you. You can use this class to create agents that can interact with the LLM model. The Agent class has the following attributes and methods:
- Attributes:
    - role_description: The role of the agent. For example, if you want this agent to be a history expert, you can set the role_description to "You are a history expert. You will only answer questions based on what really happened in the past. Do not generate any answer if you don't have reliable sources.".
    - task_description: The task of the agent. For example, if you want this agent to answer questions only in yes/no, you can set the task_description to "Please answer the following question in yes/no. Explanations are not needed."
    - llm: Just an indicator of the LLM model used by the agent.
- Method:
    - inference: This method takes a message as input and returns the generated response from the LLM model. The message will first be formatted into proper input for the LLM model. (This is where you can set some global instructions like "Please speak in a polite manner" or "Please provide a detailed explanation".) The generated response will be returned as the output.

In [43]:
class LLMAgent():
    def __init__(self, role_description: str, llm:str="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF"):
        self.role_description = role_description   # Role means who this agent should act like. e.g. the history expert, the manager......
        self.llm = llm  # LLM indicates which LLM backend this agent is using.
    def inference(self, message:str) -> str:
        if self.llm == 'bartowski/Meta-Llama-3.1-8B-Instruct-GGUF': # If using the default one.
            # TODO: Design the system prompt and user prompt here.
            # Format the messsages first.
            messages = [
                {"role": "system", "content": f"{self.role_description}"},  # Hint: you may want the agents to speak Traditional Chinese only.
                {"role": "user", "content": f"{message}"}, # Hint: you may want the agents to clearly distinguish the task descriptions and the user messages. A proper seperation text rather than a simple line break is recommended.
            ]
            return generate_response(llama3, messages)
        else:
            # TODO: If you want to use LLMs other than the given one, please implement the inference part on your own.
            return ""

TODO 1: Design the role description and task description for each agent.

In [44]:
# TODO: Design the role and task description for each agent.

# This agent may help you filter out the irrelevant parts in question descriptions.
# question_extraction_agent = LLMAgent(
#     role_description="",
# )

# This agent may help you extract the keywords in a question so that the search tool can find more accurate results.
keyword_extraction_agent = LLMAgent(
    role_description="你是 LLaMA-3.1-8B，是一個善用網絡搜尋工具的AI，簡練，沒有廢話。",
)

# This agent is the core component that answers the question.
qa_agent = LLMAgent(
    role_description="你是 LLaMA-3.1-8B，是用來回答問題的 AI。使用中文時只會使用繁體中文來回問題。",
)

## RAG pipeline

TODO 2: Implement the RAG pipeline.

Please refer to the homework description slides for hints.

Also, there might be more heuristics (e.g. classifying the questions based on their lengths, determining if the question need a search or not, reconfirm the answer before returning it to the user......) that are not shown in the flow charts. You can use your creativity to come up with a better solution!

- Naive approach (simple baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/naive.png)

- Naive RAG approach (medium baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/naive_rag.png)

- RAG with agents (strong baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/rag_agent.png)

In [45]:
import re

async def pipeline(question: str) -> str:
    # TODO: Implement your pipeline.
    # Currently, it only feeds the question directly to the LLM.
    # You may want to get the final results through multiple inferences.
    # Just a quick reminder, make sure your input length is within the limit of the model context window (16384 tokens), you may want to truncate some excessive texts.
    output = keyword_extraction_agent.inference(f"""
    基於使用者發送的消息，過濾和問題無關信息，還原代詞，並提取出最相關且有用的關鍵詞，用於進行網絡搜索。
    
    input："《王者荣耀》是腾讯天美工作室推出的英雄竞技手游，不是一个人的王者，而是团队的荣耀！5v5王者峡谷PVP对战，领略英雄竞技的酣畅淋漓！那這款遊戲现在是什么赛季？"
    thinking: 在問題前包含了很多無關背景，需要去除。這款遊戲是代詞，需要還原成“王者荣耀”。所以用戶想問“那這款遊戲现在是什么赛季？”。
    output：抽取關鍵詞：```王者荣耀 赛季```

    input：{question}""")
    match = re.search(r'```(.*?)```', output, re.DOTALL)
    add_on = ""
    retrieval_add_on = ""
    if match:
        keyword = match.group(1)
        retrieval_add_on = await search(keyword)
        add_on = f"以下內容是基於{keyword}的搜尋結果:{retrieval_add_on}"
    retrieval = ""
    retrieval = await search(question)

    answer = qa_agent.inference(f"""
    
        以下內容是基於使用者發送的消息的搜尋結果:
        {retrieval}

        {add_on}
    
        在我給你的搜尋結果中，每個結果都是[webpage X begin]...[webpage X end]格式的，X代表每篇文章的數字索引。請在適當的情況下在句子末尾引用上下文。請按照引用編號[citation:X]的格式在答案對應部分引用上下文。如果一句話源自多個上下文，請列出所有相關的引用編號，例如[citation:3][citation:5]，切記不要將引用集中在最後返回引用編號，而是在答案對應部分列出。 
        在回答時，請注意以下幾點： 
        - 今天是{cur_day}。 
        - 並非搜尋結果的所有內容都與使用者的問題密切相關，你需要結合問題，對搜尋結果進行甄別、篩選。
        - 如果回答很长，请尽量结构化、分段落总结。如果需要分点作答，尽量控制在5个点以内，并合并相关的内容。
        - 对于客观类的问答，如果问题的答案非常简短，可以适当补充一到两句相关信息，以丰富内容。
        - 你的回答应该综合多个相关网页来回答，不能重复引用一个网页。
    
        使用者訊息為：
        {question}""")
    
    return answer

## Answer the questions using your pipeline!

Since Colab has usage limit, you might encounter the disconnections. The following code will save your answer for each question. If you have mounted your Google Drive as instructed, you can just rerun the whole notebook to continue your process.

In [ ]:
from pathlib import Path

# Fill in your student ID first.
STUDENT_ID = "wust202413703008"

STUDENT_ID = STUDENT_ID.lower()
with open('./public.txt', 'r') as input_f:
    questions = input_f.readlines()
    questions = [l.strip().split(',')[0] for l in questions]
    for id, question in enumerate(questions, 1):
        # if Path(f"./{STUDENT_ID}_{id}.txt").exists():
        #     continue
        answer = await pipeline(question)
        answer = answer.replace('\n',' ')
        print(id, answer)
        with open(f'./{STUDENT_ID}_{id}.txt', 'w') as output_f:
            print(answer, file=output_f)

1 根據你的問題，我們可以在搜尋結果中找到相關的資訊。[webpage 0 begin]校歌南投縣 南 投 市 光 華 國 小 - 精華區 lyrics-批踢 蹴實業坊... [citation:1]  「虎山雄風飛揚」是光华国小（位於台湾）的一首 校 歌。
2 根據搜尋結果，2025年初NCC透過行政命令規定民眾如果通過境外郵購無線鍑盤、滑鼠等自用產品回台，每案一律加收審查費750元。[citation:0]  這項政策引發了廣大網友的不滿，認為此舉是搶錢，並非真正關心民眾安全或電波秩序。  後來行政院長卓榮泰表示NCC已決定停徵審查費，這意味著自2025年2月14日起，不再收取750元的檢驗稅。[citation:1]  然而，仍有疑慾出現，因為之前就曾經宣布暫緩，但現在又說是停止，因此有人質問未來是否還會恢復徵費。  NCC主秘黃文哲表示，這項政策主要針對代購業者，而非民眾自用，若確認屬於後者的話，就將朝免收或減少稅金的方向進行研議。[citation:2]  至今已有322位網友繳交了750元檢驗費，但NCC表示會盡快辦理退款給這些人。  綜上所述，2025年初通過境外郵購無線鍑盤、滑鼠等自用產品回台，每案一律加收審查稅金為 750 元。
3 第一代 iPhone 是由史蒂夫·乔布斯發表的。[citation:1]
4 根據台大進階英文免修申請規定，托福網路測驗(TOEFL iBT)需達到72分以上才能符合條件。[citation:0]  另外，也有提及其他相關的資訊，如：  * 外語教學暨 資源中心自訂 測試   * 全民英語能力 分級檢定 中高 級初 試（含）之上      [ citation :2 ]    托福網路測驗(TOEFL iBT)72分以上是申請進階英文免修的一個條件，但這並不是唯一的選擇，還有其他相關資訊需要考慮。
5 觸地 try 可得 5 分。根據橄欖球規則，當一名選手持有著足球並且在對方的進攻區域內將其踢在地上時，就會被判定為一個成功完成了 Try 的動作，這樣就可以獲得五分。  [citation:1][citation：2]
6 根據[webpage 1 begin]卑南族-维基百科，自由的... [citation:2][citaion：3]  答：  根据《維客》中的資料，我們可以知道 區祖先發源自ruvuwa'an。
7

In [ ]:
with open('./private.txt', 'r') as input_f:
    questions = input_f.readlines()
    for id, question in enumerate(questions, 31):
        # if Path(f"./{STUDENT_ID}_{id}.txt").exists():
        #     continue
        answer = await pipeline(question)
        answer = answer.replace('\n',' ')
        print(id, answer)
        with open(f'./{STUDENT_ID}_{id}.txt', 'a') as output_f:
            print(answer, file=output_f)

In [ ]:
# Combine the results into one file.
with open(f'./{STUDENT_ID}.txt', 'w') as output_f:
    for id in range(1,91):
        with open(f'./{STUDENT_ID}_{id}.txt', 'r') as input_f:
            answer = input_f.readline().strip()
            print(answer, file=output_f)